# **creating the dfs to store triplet chnage and counts information (average and sd)** 

In [1]:
from datetime import datetime
import numpy as np 
import pandas as pd
import json 
from tqdm import tqdm 

### **loading in the data** 

In [2]:
model = json.load(open("../Human_mutability_model/Model_2020_12_02_genomeWide.txt"))

In [3]:
ntrial = 10  
#importing data 

#DNA TRIPLET CHNAGE 
#generating the empty tripfreq disctionaries for each coverage level 
#tripfreq_dnaX = {"trip":[0,for,each,trial], "next_trip":[0,for,each,trial] ....}
tripfreq_dna0 = {}
tripfreq_dna01 = {}
tripfreq_dna2 = {}
for triplet in list(model.keys()): 
    tripfreq_dna0[triplet] = [0]*ntrial
    tripfreq_dna01[triplet] = [0]*ntrial
    tripfreq_dna2[triplet] = [0]*ntrial

#filling the empty tripfreq dictionaries 
#tripfreq_dnaX = {"trip":[count,for,each,trial], "next_trip":[count,for,each,trial] ....}
for trial in range(1, ntrial+1): 
    dna_0 = open("data/cov2x_2021_04_18_Trial{}_DNA_0.txt".format(trial)).readlines()[0]
    dna_01 = open("data/cov2x_2021_04_18_Trial{}_DNA_10000.txt".format(trial)).readlines()[0]
    dna_2 = open("data/cov2x_2021_04_18_Trial{}_DNA_199999.txt".format(trial)).readlines()[0]
    for j in range(1, len(dna_0)-2): 
        cur_trip_dna0 = dna_0[j-1:j+2]#getting the triplet for that index of the sequence 
        cur_trip_dna01 = dna_01[j-1:j+2]
        cur_trip_dna2 = dna_2[j-1:j+2]
        tripfreq_dna0[cur_trip_dna0][trial-1] += 1#adding the triple tcount to the correct index of the trial list 
        tripfreq_dna01[cur_trip_dna01][trial-1] += 1
        tripfreq_dna2[cur_trip_dna2][trial-1] += 1

#TRIPLET COUNTS INFO 
#generating the empty tripcount disctionaries for each coverage level (excpet 0 ofcourse) 
#tripcount_dnaX = {"trip":[0,for,each,trial], "next_trip":[0,for,each,trial] ....}
tripcount_chosen_01 = {}
tripcount_chosen_2 = {}
tripcount_into_01 = {}
tripcount_into_2 = {}
for triplet in list(model.keys()): 
    tripcount_chosen_01[triplet] = [0]*ntrial
    tripcount_chosen_2[triplet] = [0]*ntrial
    tripcount_into_01[triplet] = [0]*ntrial
    tripcount_into_2[triplet] = [0]*ntrial
    
#filling the empty tripfreq dictionaries 
for trial in range(1, ntrial+1): 
    cur_chosen_01 = json.load(open("data/cov2x_2021_04_18_Trial{}_triplet_chosen_count_dict_10000.txt".format(trial)))
    cur_chosen_2 = json.load(open("data/cov2x_2021_04_18_Trial{}_triplet_chosen_count_dict_199999.txt".format(trial)))
    cur_into_01 = json.load(open("data/cov2x_2021_04_18_Trial{}_triplet_into_count_dict_10000.txt".format(trial)))
    cur_into_2 = json.load(open("data/cov2x_2021_04_18_Trial{}_triplet_into_count_dict_199999.txt".format(trial)))
    for triplet in list(model.keys()): 
        tripcount_chosen_01[triplet][trial-1] = cur_chosen_01[triplet][1] #adding the value for the cur trail dict to the overall tripcount dict 
        tripcount_chosen_2[triplet][trial-1] = cur_chosen_2[triplet][1]#need the [0,1,2] middle spot as that represents the central triplet in the dict 
        tripcount_into_01[triplet][trial-1] = cur_into_01[triplet][1]#need trial-1 as trial 1-->10 and index of the list 0-->9 
        tripcount_into_2[triplet][trial-1] = cur_into_2[triplet][1]

In [6]:
out_file = open("data/Data_dataframes/dict_2021_04_18_2xCov_triplet_counts.txt", "w")
json.dump(tripfreq_dna2, out_file, indent = 6)
out_file.close()

out_file = open("data/Data_dataframes/dict_2021_04_18_0.1xCov_triplet_counts.txt", "w")
json.dump(tripfreq_dna01, out_file, indent = 6)
out_file.close()

### **need to analyze and convert into sensible data for the dataframe** 

In [4]:
#need a dictionary like: 
#{"triplet":[mut, earlyChange, LateChnage, earlyChosen,earlyInto,lateChosen,LateInto]}

data_dict = {}
for triplet, mutVal in model.items(): 
    mut = mutVal[0]
    mut_log10 = np.log10(mut)
    
    #claculate the chnage for each trial, then average 
    early_change = []
    late_change = []
    for i in range(ntrial): 
        early_change.append((tripfreq_dna01[triplet][i]-tripfreq_dna0[triplet][i])/tripfreq_dna0[triplet][i])
        late_change.append((tripfreq_dna2[triplet][i]-tripfreq_dna0[triplet][i])/tripfreq_dna0[triplet][i])
    early_chnage_avg = np.mean(early_change)
    early_chnage_stderr = np.std(early_change)/ntrial
    late_chnage_avg = np.mean(late_change)
    late_chnage_stderr = np.std(late_change)/ntrial
    
    #get the triplet counts 
    early_counts_chosen_avg = np.mean(tripcount_chosen_01[triplet])
    early_counts_chosen_stderr = np.std(tripcount_chosen_01[triplet])/ntrial
    early_counts_into_avg = np.mean(tripcount_into_01[triplet])
    early_counts_into_stderr = np.std(tripcount_into_01[triplet])/ntrial
    #now for the late (2x coverage) 
    late_counts_chosen_avg = np.mean(tripcount_chosen_2[triplet])
    late_counts_chosen_stderr = np.std(tripcount_chosen_2[triplet])/ntrial
    late_counts_into_avg = np.mean(tripcount_into_2[triplet])
    late_counts_into_stderr = np.std(tripcount_into_2[triplet])/ntrial
    
    #making the dict
    data_dict[triplet] = [triplet, mut, mut_log10,
                          early_chnage_avg, early_chnage_stderr, 
                          late_chnage_avg, late_chnage_stderr, 
                          early_counts_chosen_avg, early_counts_chosen_stderr, early_counts_into_avg, early_counts_into_stderr,
                          late_counts_chosen_avg, late_counts_chosen_stderr, late_counts_into_avg, late_counts_into_stderr] 

# **adding the adjacent and "into" probabilities**

# **creating df**

In [5]:
plotting_df = pd.DataFrame.from_dict(data_dict, orient = "index", columns=["triplet", "mut", "mut_log10",
                                                                           "change_01_mean", "change_early_stder", 
                                                                           "change_2_mean","change_2_stder",
                                                                           "countsChosen_01_mean", "countsChosen_01_stder", "countsInto_01_mean", "countsInto_01_stder",
                                                                           "countsChosen_2_mean", "countsChosen_2_stder", "countsInto_2_mean","countsInto_2_stder"])

In [6]:
plotting_df.head()

,triplet,mut,mut_log10,change_01_mean,change_early_stder,change_2_mean,change_2_stder,countsChosen_01_mean,countsChosen_01_stder,countsInto_01_mean,countsInto_01_stder,countsChosen_2_mean,countsChosen_2_stder,countsInto_2_mean,countsInto_2_stder
TGC,TGC,0.000030,-4.523276,0.152708,0.001290,-0.165829,0.001888,107.3,1.059292,86.5,0.972882,2829.1,4.954886,2604.6,4.115872
GCG,GCG,0.000293,-3.533051,-0.680967,0.001388,-0.938362,0.000845,551.6,1.435409,82.6,0.637495,2731.5,4.436271,1965.0,4.531666
CGG,CGG,0.000284,-3.546064,-0.646009,0.000668,-0.938221,0.000574,577.3,1.951948,96.5,0.576628,2744.0,4.376985,2220.9,4.658852
AGA,AGA,0.000030,-4.529189,0.123403,0.001777,0.330322,0.004149,97.5,1.188486,63.9,0.338969,3891.4,7.145236,3173.0,4.499333
GTT,GTT,0.000021,-4.681856,0.021478,0.001849,0.372329,0.002323,64.8,1.018627,134.0,1.218195,2683.8,6.229414,3448.9,6.014890


In [49]:
plotting_df.to_csv("data/Data_dataframes/final_plotting_01_2x_coverage_2021_04_21.csv")